In [1]:
import csv
import json
import numpy as np
import os

from collections import Counter

In [2]:
from IPython.display import clear_output
from IPython.core.display import HTML
HTML("<style>.container { width:98% !important; }</style>")

In [3]:
if os.path.basename(os.path.normpath(os.getcwd())) != "fast2":
    os.chdir('..')
print(os.getcwd())

/g/g92/noh1/fast2


In [4]:
import metrics

METRICS = {
    "rmse": metrics.RMSE,
    "mae": metrics.MAE,
    "r2": metrics.R2,
    "pearson_r": metrics.PearsonR,
    "spearman_r": metrics.SpearmanR,
}

metric_names = ["rmse", "mae", "r2", "pearson_r", "spearman_r"]

/g/g92/noh1/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
result_dir = "./results"

exp_names = [
#     "PDBBIND2020/PCN-20230426-114729-clycx2fc-GENERAL+REFINED",
#     "PDBBIND2020/SGCNN-20230426-114626-2xm6bzhz-GENERAL+REFINED",
#     "MPRO/PCN-20230426-180113-rqn56jfo-TRAIN80",
#     "MPRO/PCN-20230426-220511-jlme00lo-TRAIN80_FINETUNE",
#     "MPRO/SGCNN-20230426-180233-c2_c__zb-TRAIN80",
#     "MPRO/SGCNN-20230426-220143-9yu8ht5m-TRAIN80_FINETUNE",
#     "MPRO/PCN-20230426-200743-4an40ysz-TRAIN80_RANDOM",
#     "MPRO/PCN-20230426-220353-ssrgp_2_-TRAIN80_RANDOM_FINETUNE",
#     "MPRO/SGCNN-20230427-114553-2wfput89-TRAIN80_RANDOM",
#     "MPRO/SGCNN-20230426-220002-u92tdy7q-TRAIN80_RANDOM_FINETUNE",
#     "MPRO+PDBBIND2020/PCN-20230502-221454-2mhg9kyx-SCENARIO4",
#     "MPRO+PDBBIND2020/PCN-20230503-000641-66q9v6qy-SCENARIO4_RANDOM",
#     "MPRO+PDBBIND2020/SGCNN-20230502-221429-r_d4z77j-SCENARIO4",
#     "MPRO+PDBBIND2020/SGCNN-20230503-000753-lh8lmz3v-SCENARIO4_RANDOM",
    "MPRO/FUSION_SGCNN+EFFICIENTSE3-20230720-202416-nr3676x1-FREEZE_ATT",
    "MPRO/FUSION_SGCNN+EFFICIENTSE3-20230720-202716-3ty9h81l-FREEZE_ATT_x0.1",
    "MPRO/FUSION_SGCNN+EFFICIENTSE3-20230720-203059-b7s37aog-FREEZE_WO_ATT",
    "MPRO/FUSION_SGCNN+EFFICIENTSE3-20230720-202913-gpfdo3ls-FREEZE_WO_ATT_x0.1",
    "MPRO/FUSION_PCN+EFFICIENTSE3-20230720-204800-waizt831-FREEZE_ATT",
    "MPRO/FUSION_SGCNN+EFFICIENTSE3-20230720-202716-3ty9h81l-FREEZE_ATT_x0.1",
    "MPRO/FUSION_PCN+EFFICIENTSE3-20230720-205151-igm3s3iu-FREEZE_WO_ATT",
    "MPRO/FUSION_PCN+EFFICIENTSE3-20230720-205052-t1pynugy-FREEZE_WO_ATT_x0.1",
]


print("total exps:", len(exp_names))

overall_predictions = {}

for exp_name in exp_names:
    exp_dir = os.path.join(result_dir, exp_name)
    sub_dirs = [name for name in os.listdir(exp_dir) if (os.path.isdir(os.path.join(exp_dir, name)) and ("mpro" in name))]
    for test_set in sub_dirs:
        pred_path = os.path.join(exp_dir, test_set, "predictions.json")
        if os.path.exists(pred_path):
            print(pred_path)
            with open(pred_path, 'r') as f:
                predictions = json.load(f)
            overall_predictions[exp_name + "/" + test_set] = predictions
        else:
            print("Not exists: {}".format(pred_path))

total exps: 8
./results/MPRO/FUSION_SGCNN+EFFICIENTSE3-20230720-202416-nr3676x1-FREEZE_ATT/mpro_test/predictions.json
./results/MPRO/FUSION_SGCNN+EFFICIENTSE3-20230720-202716-3ty9h81l-FREEZE_ATT_x0.1/mpro_test/predictions.json
./results/MPRO/FUSION_SGCNN+EFFICIENTSE3-20230720-203059-b7s37aog-FREEZE_WO_ATT/mpro_test/predictions.json
./results/MPRO/FUSION_SGCNN+EFFICIENTSE3-20230720-202913-gpfdo3ls-FREEZE_WO_ATT_x0.1/mpro_test/predictions.json
./results/MPRO/FUSION_PCN+EFFICIENTSE3-20230720-204800-waizt831-FREEZE_ATT/mpro_test/predictions.json
./results/MPRO/FUSION_SGCNN+EFFICIENTSE3-20230720-202716-3ty9h81l-FREEZE_ATT_x0.1/mpro_test/predictions.json
./results/MPRO/FUSION_PCN+EFFICIENTSE3-20230720-205151-igm3s3iu-FREEZE_WO_ATT/mpro_test/predictions.json
./results/MPRO/FUSION_PCN+EFFICIENTSE3-20230720-205052-t1pynugy-FREEZE_WO_ATT_x0.1/mpro_test/predictions.json


In [6]:
def get_compound_id(docking_id):
    return docking_id.rsplit("_", 2)[0]

method_names = ["best", "highest", "average"]
poses = list(range(1, 11))

print(" ".join(["model", "method"] + metric_names))
for exp_name, predictions in overall_predictions.items():
    dock_ids = list(predictions.keys())
    comp_ids = list(set([get_compound_id(dock_id) for dock_id in dock_ids]))
#     print("{}: # docking = {} / # compound = {}".format(exp_name, len(dock_ids), len(comp_ids)))

    # All
    all_dts = np.array([pred for pred in predictions.values()])[:, 0]
    all_gts = np.array([pred for pred in predictions.values()])[:, 1]
          
    row = [exp_name, "all"] + [METRICS[metric_name]().calculate(all_dts, all_gts) for metric_name in metric_names]
    row = [str(r) for r in row]
    print(" ".join(row))
          
    dts = {method_name: [] for method_name in method_names}
    gts = []

    for comp_id in comp_ids:
        _dts = [predictions["{}_protease_{}".format(comp_id, str(pose))][0] for pose in poses]
        _gts = [predictions["{}_protease_{}".format(comp_id, str(pose))][1] for pose in poses]

        assert len(set(_gts))

        dts["best"].append(_dts[0])
        dts["highest"].append(max(_dts))
        dts["average"].append(sum(_dts) / len(_dts))
        gts.append(_gts[0])

    gts = np.array(gts)
    for method_name in method_names:
        _dts = np.array(dts[method_name])
        row = [exp_name, method_name] + [METRICS[metric_name]().calculate(_dts, gts) for metric_name in metric_names]
        row = [str(r) for r in row]
        print(" ".join(row))
        
#     print()

model method rmse mae r2 pearson_r spearman_r
MPRO/FUSION_SGCNN+EFFICIENTSE3-20230720-202416-nr3676x1-FREEZE_ATT/mpro_test all 0.8694438376360641 0.7176371516871618 -0.11606911280592835 nan nan
MPRO/FUSION_SGCNN+EFFICIENTSE3-20230720-202416-nr3676x1-FREEZE_ATT/mpro_test best 0.8694438376360641 0.7176371516871618 -0.11606911280592835 nan nan
MPRO/FUSION_SGCNN+EFFICIENTSE3-20230720-202416-nr3676x1-FREEZE_ATT/mpro_test highest 0.8694438376360641 0.7176371516871618 -0.11606911280592835 nan nan
MPRO/FUSION_SGCNN+EFFICIENTSE3-20230720-202416-nr3676x1-FREEZE_ATT/mpro_test average 0.8694438376360641 0.7176371516871618 -0.11606911280592835 nan nan
MPRO/FUSION_SGCNN+EFFICIENTSE3-20230720-202716-3ty9h81l-FREEZE_ATT_x0.1/mpro_test all 0.8695312190561836 0.717696850371748 -0.1162934599126706 nan nan
MPRO/FUSION_SGCNN+EFFICIENTSE3-20230720-202716-3ty9h81l-FREEZE_ATT_x0.1/mpro_test best 0.8695312190561836 0.717696850371748 -0.11629345991267082 nan nan
MPRO/FUSION_SGCNN+EFFICIENTSE3-20230720-202716-3t

/g/g92/noh1/.local/lib/python3.8/site-packages/scipy/stats/_stats_py.py:4424: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))
/g/g92/noh1/.local/lib/python3.8/site-packages/scipy/stats/_stats_py.py:4921: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(warn_msg))
/g/g92/noh1/.local/lib/python3.8/site-packages/scipy/stats/_stats_py.py:4424: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))
/g/g92/noh1/.local/lib/python3.8/site-packages/scipy/stats/_stats_py.py:4921: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(warn_msg))
/g/g92/noh1/.local/lib/python3.8/site-packages/scipy/stats/_stats_py.py:4424: ConstantInputWarning: An inp